Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (50000, 28, 28) (50000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (50000, 784) (50000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [36]:
batch_size = 128
num_hidden_nodes= 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  def model(data):
    first_layer = tf.matmul(data, weights_1) + biases_1
    first_relu = tf.nn.relu(first_layer)
    second_layer = tf.matmul(first_relu, weights_2) + biases_2
    return second_layer
    
    
  logits = model(tf_train_dataset)
  reg_mult = 1e-3
  regularization = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(biases_2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) +reg_mult*regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [37]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 690.219177
Minibatch accuracy: 9.4%
Validation accuracy: 40.7%
Minibatch loss at step 500: 192.256851
Minibatch accuracy: 84.4%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 116.600121
Minibatch accuracy: 85.9%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 69.037132
Minibatch accuracy: 89.8%
Validation accuracy: 83.1%
Minibatch loss at step 2000: 42.036819
Minibatch accuracy: 89.1%
Validation accuracy: 84.5%
Minibatch loss at step 2500: 25.409475
Minibatch accuracy: 95.3%
Validation accuracy: 84.8%
Minibatch loss at step 3000: 15.536762
Minibatch accuracy: 94.5%
Validation accuracy: 85.7%
Test accuracy: 91.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [44]:
batch_size =128
num_hidden_nodes= 1024

restricted_train_dataset = train_dataset[0:batch_size*3,:]
restricted_train_labels = train_labels[0:batch_size*3,:]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  def model(data):
    first_layer = tf.matmul(data, weights_1) + biases_1
    first_relu = tf.nn.relu(first_layer)
    second_layer = tf.matmul(first_relu, weights_2) + biases_2
    return second_layer
    
    
  logits = model(tf_train_dataset)
  reg_mult = 0
  regularization = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(biases_2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) +reg_mult*regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [67]:
num_steps = 3001

with tf.Session(config=tf.ConfigProto(log_device_placement=True), graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (restricted_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = restricted_train_dataset[offset:(offset + batch_size), :]
    batch_labels = restricted_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 7.330859
Minibatch accuracy: 10.9%
Validation accuracy: 13.5%
Minibatch loss at step 500: 3.867458
Minibatch accuracy: 64.8%
Validation accuracy: 73.6%
Minibatch loss at step 1000: 2.612460
Minibatch accuracy: 64.1%
Validation accuracy: 74.8%
Minibatch loss at step 1500: 2.069078
Minibatch accuracy: 57.8%
Validation accuracy: 75.6%
Minibatch loss at step 2000: 1.428511
Minibatch accuracy: 62.5%
Validation accuracy: 76.4%
Minibatch loss at step 2500: 1.394773
Minibatch accuracy: 53.1%
Validation accuracy: 76.5%
Minibatch loss at step 3000: 1.188869
Minibatch accuracy: 57.8%
Validation accuracy: 76.6%
Test accuracy: 82.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [57]:
batch_size =128
num_hidden_nodes= 1024

restricted_train_dataset = train_dataset[0:batch_size*3,:]
restricted_train_labels = train_labels[0:batch_size*3,:]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes], stddev=0.1))
  weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels], stddev=0.1))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  def model(data, train):
    first_layer = tf.matmul(data, weights_1) + biases_1
    first_relu = tf.nn.relu(first_layer)
    if (train):
        first_relu = tf.nn.dropout(first_relu, 0.5)
    second_layer = tf.matmul(first_relu, weights_2) + biases_2
    return second_layer
    
    
  logits = model(tf_train_dataset, True)
  reg_mult = 1e-3
  regularization = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(biases_2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) +reg_mult*regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, False))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, False))

In [59]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.331120
Minibatch accuracy: 11.7%
Validation accuracy: 38.0%
Minibatch loss at step 500: 0.633086
Minibatch accuracy: 84.4%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.597534
Minibatch accuracy: 84.4%
Validation accuracy: 85.5%
Minibatch loss at step 1500: 0.581651
Minibatch accuracy: 85.2%
Validation accuracy: 86.3%
Minibatch loss at step 2000: 0.625408
Minibatch accuracy: 86.7%
Validation accuracy: 87.0%
Minibatch loss at step 2500: 0.451591
Minibatch accuracy: 85.9%
Validation accuracy: 86.8%
Minibatch loss at step 3000: 0.526192
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 3500: 0.459789
Minibatch accuracy: 90.6%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 0.498885
Minibatch accuracy: 89.1%
Validation accuracy: 86.8%
Minibatch loss at step 4500: 0.543057
Minibatch accuracy: 89.1%
Validation accuracy: 87.5%
Minibatch loss at step 5000: 0.593384
Minibatch accuracy: 86.7%
Validation accurac

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [62]:
batch_size =128
num_hidden_nodes= 1024
num_second_hidden_nodes = 300
num_third_hidden_nodes = 50

restricted_train_dataset = train_dataset[0:batch_size*3,:]
restricted_train_labels = train_labels[0:batch_size*3,:]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes], stddev=0.1))
  weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_second_hidden_nodes], stddev=0.1))
  weights_3 = tf.Variable(tf.truncated_normal([num_second_hidden_nodes, num_third_hidden_nodes], stddev=0.1))
  weights_4 = tf.Variable(tf.truncated_normal([num_third_hidden_nodes, num_labels], stddev=0.1))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  biases_2 = tf.Variable(tf.zeros([num_second_hidden_nodes]))
  biases_3 = tf.Variable(tf.zeros([num_third_hidden_nodes]))
  biases_4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  def model(data, train):
    first_layer = tf.matmul(data, weights_1) + biases_1
    first_relu = tf.nn.relu(first_layer)
    second_layer = tf.matmul(first_relu, weights_2) + biases_2
    second_relu = tf.nn.relu(second_layer)
    third_layer = tf.matmul(second_relu, weights_3) + biases_3
    third_relu = tf.nn.relu(third_layer)
    fourth_layer = tf.matmul(third_relu, weights_4) + biases_4
    if (train):
        fourth_layer = tf.nn.dropout(fourth_layer, 0.5)
    
    return fourth_layer
    
    
  logits = model(tf_train_dataset, True)
  reg_mult = 1e-3
  regularization = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4)
  regularization = regularization + tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(biases_2) + tf.nn.l2_loss(biases_3) + tf.nn.l2_loss(biases_4)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) +reg_mult*regularization
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, len(train_dataset), 0.95)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, False))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, False))

In [64]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 7.843565
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 500: 5.163994
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 4.027102
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 2.297109
Minibatch accuracy: 52.3%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 1.917764
Minibatch accuracy: 48.4%
Validation accuracy: 85.1%
Minibatch loss at step 2500: 1.648272
Minibatch accuracy: 47.7%
Validation accuracy: 85.3%
Minibatch loss at step 3000: 1.473447
Minibatch accuracy: 50.0%
Validation accuracy: 85.0%
Minibatch loss at step 3500: 1.100879
Minibatch accuracy: 64.1%
Validation accuracy: 86.8%
Minibatch loss at step 4000: 1.254509
Minibatch accuracy: 50.0%
Validation accuracy: 86.7%
Minibatch loss at step 4500: 1.230649
Minibatch accuracy: 57.0%
Validation accuracy: 86.9%
Minibatch loss at step 5000: 1.191675
Minibatch accuracy: 54.7%
Validation accuracy